In [14]:
import os

from pytube import Playlist
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Neo4jVector
from langchain.document_loaders import YoutubeLoader

from langchain.chat_models import ChatOpenAI
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [15]:
# playlist_url = "https://www.youtube.com/watch?v=hCMv4UJo--4&list=PL9Hl4pk2FsvWPcphew_GbLjCWvMpmh4mV&ab_channel=Neo4j"
# playlist = Playlist(playlist_url)
# video_ids = [_v.split('v=')[-1] for _v in playlist.video_urls]
# Process only one video
video_ids = ['qYaXwV9ZNr4']

In [16]:
# Init text splitter with chunk size 512 (https://www.pinecone.io/learn/chunking-strategies/)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=20)

In [17]:
from env import setup_env
setup_env()

In [18]:
# Read their captions and process it into documents with above defined text splitter
documents = []
for video_id in video_ids:
    loader = YoutubeLoader(video_id=video_id)
    documents.append(loader.load_and_split(text_splitter=text_splitter)[0])

In [19]:
# Contruct vector
neo4j_vector = Neo4jVector.from_documents(
    embedding=OpenAIEmbeddings(),
    documents=documents,
    url=os.environ['NEO4J_URI'],
    username=os.environ['NEO4J_USERNAME'],
    password=os.environ['NEO4J_PASSWORD']
)

In [21]:
# Prepare Q&A object
chat = ChatOpenAI()
chat_mem_history = ChatMessageHistory(sesion_id="1")
mem = ConversationBufferMemory(memory_key="chat_history", chat_memory=chat_mem_history, return_messages=True)
q = ConversationalRetrievalChain.from_llm(
    llm=chat,
    memory=mem,
    retriever=neo4j_vector.as_retriever()
)

In [23]:
response = q({'question': 'Hi'})
response.get('answer')

In [8]:
query = ""
results = neo4j_vector.similarity_search(query, k=1)
print(results[0].page_content)

in 2023 in Pro Road cycling there were team yimu visma and the rest this is going to change the team who won for the first time in history all three grand Tours in the same year and with different Riders is gone it is official that by the end of 2023 team yumbo visma will cease to exist what is the future for yunas vingard VA vanard seus or Mariana F like the fix the team will be born again under a new [Music] name hello I'm Gabriel welcome to cycling after 14 following a historic season containing an unprecedented three grand tour victories team yumbo visma will continue as team visma leads a bike after 5 years as second title sponsor visma will be the team's first title sponsor from the 2024 season so who is visma and who is Lisa bike visma was founded in Norway in 1996 since then they have grown into Europe's number one Cloud software company operating throughout Europe and Latin America visma offers software that simplify and digitize processes for businesses and organizations in t